In [1]:
import pandas as pd
import torch
import re
import ast
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = load_dataset('saracandu/my_wikihop', split='train')
sources = dataset['supports']

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", use_fast = False)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
def produce_prompt(source):
    messages = [ 
        {"role": "system", "content": """
        You are a helpful AI assistant. You are given a long document as context and you are asked to produce a meaningful summary of it.
        Please summarize in less than 500 words.
        """}, 
        {"role": "user", "content": source},
    ] 
    return messages

pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 750, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

In [11]:
summ = []
for i in range(10):
    output = pipe(produce_prompt(sources[i]), **generation_args) 
    summ.append(output[0]['generated_text']) 
    torch.cuda.empty_cache()

In [12]:
summ[9]

" Canada, the second-largest country by total area and the fourth-largest by land area, is located in the northern half of North America. It extends from the Atlantic to the Pacific and northward into the Arctic Ocean, covering a vast territory of over 9.98 million square kilometers. The country's diverse climate ranges from cold or severely cold winters in the majority of its land territory to warm summers in the southerly areas.\n\nCanada's population of 36 million is predominantly urbanized, with about four-fifths living near the southern border. The country's capital is Ottawa, and its largest city is Toronto. Other major urban areas include Montreal, Vancouver, Calgary, Edmonton, Quebec City, Winnipeg, and Hamilton.\n\nCanadian music has been influenced by various genres, including alternative country, indie pop, pop rock, folk, and indie rock. Justin Rutledge, Jill Barber, and Matthew Barber are notable Canadian singer-songwriters who have contributed to these genres.\n\nOutside 